In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [4]:
!pip install -r CLIP/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [5]:
!pip install CLIP/.

Processing ./CLIP
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=0974ba32b2fe7889870e0763deb407c2a0e96ccc39d8a9a13d2443c0a4ab3e76
  Stored in directory: /tmp/pip-ephem-wheel-cache-kat70c4l/wheels/4e/0a/42/0c7958e3f48e9a0cc115a24613d47fb1f331b2d8e7954ec87f
Successfully built clip


In [6]:
import os
import torch
import clip
from PIL import Image
import logging
import time
import pandas as pd

# Setup logging
logging.basicConfig(filename='/content/drive/MyDrive/hotel_data/clip_tagging.log',
                    level=logging.INFO, format='%(asctime)s - %(message)s')

In [ ]:
# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

generated_texts = [
    # Rooms & Room Types
    "room", "view", "bathroom", "bedroom", "suite", "king", "queen", "twin", 
    "single", "double", "bed", "couch", "sofa", "studio", "penthouse",
    
    # Room Features & Furniture
    "balcony", "patio", "terrace", "desk", "chair", "lamp", "nightstand", 
    "closet", "wardrobe", "safe", "television", "TV", "minibar", "refrigerator", 
    "microwave", "coffee", "tea", "curtains", "window", "door", "mirror",
    
    # Bathroom Features
    "shower", "bathtub", "jacuzzi", "toilet", "sink", "hairdryer", "toiletries", 
    "amenities", "towels", "steam", "sauna",
    
    # Bedding & Comfort
    "linens", "pillow", "blanket", "duvet", "comforter", "mattress", "memory foam", 
    "pillow-top", "firm", "soft", "plush",
    
    # Hotel Areas
    "lobby", "reception", "concierge", "entrance", "corridor", "hallway", "elevator", 
    "stairs", "lounge", "courtyard", "rooftop",
    
    # Dining & Food
    "restaurant", "bar", "breakfast", "lunch", "dinner", "brunch", "buffet", 
    "à la carte", "menu", "chef", "cuisine", "meal", "room service", "dining", 
    "table", "seating",
    
    # Specialty Diets
    "vegetarian", "vegan", "gluten-free", "organic", "allergen-friendly",
    
    # Beverages
    "alcohol", "wine", "beer", "cocktail", "minibar", "water", "beverage", "ice", 
    "coffee", "tea",
    
    # Amenities & Facilities
    "pool", "gym", "spa", "fitness", "business center", "conference", "meeting", 
    "banquet", "event", "wedding", "WiFi", "internet", "laundry", "dry cleaning", 
    "parking", "valet",
    
    # Pool Types
    "indoor pool", "outdoor pool", "infinity pool", "heated pool", "lap pool", 
    "kiddie pool",
    
    # Wellness & Recreation
    "hot tub", "sauna", "steam room", "massage", "treatment", "facial", "manicure", 
    "pedicure", "weights", "treadmill", "elliptical", "bike", "yoga",
    
    # Technology & Electronics
    "WiFi", "USB", "outlet", "charging", "HDMI", "streaming", "cable", "satellite", 
    "premium channels", "Netflix", "smart room", "digital key", "app-controlled", 
    "Bluetooth", "speaker", "sound system",
    
    # Climate Control
    "air-conditioning", "heating", "fan", "blackout",
    
    # Views & Locations
    "ocean", "mountain", "city", "beach", "garden", "waterfront", "lakeside", 
    "riverside", "downtown", "suburban", "rural", "urban", "central", "panoramic", 
    "scenic", "picturesque",
    
    # Transportation & Access
    "airport shuttle", "transportation", "rental", "car", "bicycle", "walking distance", 
    "subway", "metro", "bus", "train", "station", "airport", "taxi", "uber", "lyft",
    
    # Design & Style
    "decor", "modern", "classic", "luxury", "budget", "historic", "contemporary", 
    "minimalist", "rustic", "industrial", "tropical", "Mediterranean", "alpine", 
    "colonial", "Victorian", "Art Deco",
    
    # Hotel Types
    "boutique", "chain", "independent", "resort", "motel", "inn", "lodge",
    
    # Special Features
    "family-friendly", "adults-only", "pet-friendly", "accessible", "handicap", 
    "wheelchair", "non-smoking", "smoking", "child-friendly", "kids club",
    
    # Service
    "service", "staff", "turndown", "housekeeping", "concierge", "valet", 
    "wake-up call", "towel service",
    
    # Activities & Entertainment
    "playground", "games", "activities", "entertainment", "live music", "DJs", 
    "shows", "performances", "nightlife", "clubbing", "dancing", "casino", 
    "gambling", "library", "reading area",
    
    # Water Activities
    "private beach", "cabana", "lounger", "umbrella", "sunbed", "sunscreen", 
    "poolside", "diving board", "waterslide", "water sports", "sailing", "surfing", 
    "paddleboarding", "kayaking", "jet skiing", "fishing",
    
    # Sports & Recreation
    "golf", "tennis", "basketball", "volleyball", "billiards", "ping pong", 
    "foosball", "arcade", "board games",
    
    # Special Occasions
    "honeymoon", "anniversary", "birthday", "celebration",
    
    # Atmosphere & Quality
    "privacy", "quiet", "noisy", "busy", "secluded", "isolated", "connected", 
    "convenience", "cozy", "spacious", "compact", "intimate", "expansive", 
    "clean", "fresh", "spotless", "immaculate", "well-maintained", "renovated", 
    "updated", "new", "breathtaking", "stunning", "sunset", "sunrise", "fireplace"
]

# Load image metadata from CSV
csv_path = "/content/drive/MyDrive/hotel_ibs/images.csv"
df = pd.read_csv(csv_path)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 56.6MiB/s]


In [9]:
def tag_image(image_path):
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)

        start_time = time.time()
        text_features = model.encode_text(clip.tokenize(generated_texts).to(device))
        similarities = (image_features @ text_features.T).softmax(dim=-1)
        end_time = time.time()
        print(f"Time taken to find tags: {end_time - start_time} seconds")

        top_indices = similarities[0].argsort(descending=True)[:10]  # Top 10 tags
        relevant_tags = {generated_texts[i]: float(similarities[0][i]) for i in top_indices}

        return relevant_tags
    except Exception as e:
        logging.info(f"Error tagging image {image_path}: {e}")
        return {}

def process_images_from_csv():
    results = []
    for _, row in df.iterrows():
        image_id = row["image_id"]
        image_path = row["image_url"][1:]

        # Adjust path if necessary
        image_path = f"/content/drive/MyDrive/hotel_ibs{image_path}"

        if os.path.exists(image_path):
            tag_results = tag_image(image_path)

            if tag_results:
                try:
                    logging.info(f"Tagged image {image_id}: {tag_results}")
                    print(f"Tagged {image_id}: {tag_results}")
                    results.append({"image_id": image_id, "tags": tag_results})
                except Exception as e:
                    print(e)
            else:
                print(f"Skipping {image_id} due to errors.")
        else:
            print(f"Image not found: {image_path}")

    # Save tagging results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("/content/drive/MyDrive/hotel_ibs/tagged_images.csv", index=False)
    print("Tagging results saved to Google Drive.")

In [10]:
process_images_from_csv()

Time taken to find tags: 0.4430198669433594 seconds
Tagged 2587086: {'hotel': 0.5966796875, 'room': 0.1251220703125, 'checkout': 0.08868408203125, 'suite': 0.05816650390625, 'luxury': 0.0242462158203125, 'rental': 0.0200958251953125, 'housekeeping': 0.00891876220703125, 'beds': 0.00775146484375, 'credit': 0.00705718994140625, 'business': 0.00662994384765625}
Time taken to find tags: 0.1728532314300537 seconds
Tagged 2587087: {'hotel': 0.43701171875, 'room': 0.2452392578125, 'rental': 0.0771484375, 'suite': 0.07476806640625, 'checkout': 0.057342529296875, 'luxury': 0.043975830078125, 'hostel': 0.0070648193359375, 'decor': 0.005767822265625, 'credit': 0.004352569580078125, 'apartment': 0.00360870361328125}
Time taken to find tags: 0.15787482261657715 seconds
Tagged 2587089: {'design': 0.22509765625, 'privacy': 0.1302490234375, 'checkout': 0.114990234375, 'toilet': 0.1131591796875, 'luxury': 0.1063232421875, 'rental': 0.06451416015625, 'credit': 0.04791259765625, 'decor': 0.02645874023437

In [12]:
!nvidia-smi

Fri Feb 28 17:11:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             29W /   70W |    5678MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
torch.cuda.empty_cache()